# Introduction to Spark

In this notebook, you will be introduced to the Apache Spark libary for big data processing. There's a now a python package called `pyspark` which will load Spark for you. Tthe variable `sc` is a Spark context that lets you interact with the Spark runtime. Check that it is correctly initialised:

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)
print("Ready to go!")

<SparkContext master=local[*] appName=pyspark-shell>
Ready to go!


### Learning activity: Create RDDs

To analyse large datasets using Spark you will load them into Resilient Distributed Datasets (RDDs). There are a number of ways in which you can create RDDs. Use the `parallelize()` function to create one from a Python collection of elements ["Hello", "World", "!"] 

In [2]:
helloWorld = sc.parallelize(["Hello", "World", "!"])

Use the `textFile()` function to create an RDD from the file `war-and-peace.txt` and store it in a variable called `lines`. 

In [3]:
lines = sc.textFile('war-and-peace.txt')

In [5]:
helloWorld.count()

3

### Learning activity: Basic RDD manipulation

Print the number of lines in War and Peace using the functions `count()`.

In [6]:
lines.count()

54223

Print the first 15 lines using the functions `take()`.

In [7]:
lines.take(15)

['                                      1869',
 '                                 WAR AND PEACE',
 '                                 by Leo Tolstoy',
 'BK1',
 '                                 BOOK ONE: 1805',
 'BK1|CH1',
 '  CHAPTER I',
 '',
 '  "Well, Prince, so Genoa and Lucca are now just family estates of the',
 "Buonapartes. But I warn you, if you don't tell me that this means war,",
 'if you still try to defend the infamies and horrors perpetrated by',
 'that Antichrist- I really believe he is Antichrist- I will have',
 'nothing more to do with you and you are no longer my friend, no longer',
 "my 'faithful slave,' as you call yourself! But how do you do? I see",
 'I have frightened you- sit down and tell me all the news."']

In [8]:
lines.take.__doc__

"\n        Take the first num elements of the RDD.\n\n        It works by first scanning one partition, and use the results from\n        that partition to estimate the number of additional partitions needed\n        to satisfy the limit.\n\n        Translated from the Scala implementation in RDD#take().\n\n        .. note:: this method should only be used if the resulting array is expected\n            to be small, as all the data is loaded into the driver's memory.\n\n        >>> sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)\n        [2, 3]\n        >>> sc.parallelize([2, 3, 4, 5, 6]).take(10)\n        [2, 3, 4, 5, 6]\n        >>> sc.parallelize(range(100), 100).filter(lambda x: x > 90).take(3)\n        [91, 92, 93]\n        "

### Learning activity: `filter()` and `map()` and `distinct()`

Lets apply some transformations onto RDDs. The following helper function will be useful to select the words from a line.

In [9]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

print(get_words("This, is a test!"))

['This', 'is', 'a', 'test']


Use `filter()` to count the number of lines which mention `war` and the number of lines which mention `peace`.

In [41]:
# How often are war and peace mentioned?
warFilteredCaseSensitive = lines.filter(lambda line: 'war' in line)
peaceFilteredCaseSensitive = lines.filter(lambda line: 'peace' in line)
warFiltered = lines.filter(lambda line: 'war' in line.lower())
peaceFiltered = lines.filter(lambda line: 'peace' in line.lower())

warFrequencyCaseSensitive = warFilteredCaseSensitive.count()
peaceFrequencyCaseSensitive = peaceFilteredCaseSensitive.count()
warFrequency = warFiltered.count()
peaceFrequency = peaceFiltered.count()
print("war: {} times ({} times ignoring case)".format(warFrequencyCaseSensitive, warFrequency))
print("peace: {} times ({} times ignoring case)".format(peaceFrequencyCaseSensitive, peaceFrequency))

war: 1227 times (1255 times ignoring case)
peace: 128 times (131 times ignoring case)


In [30]:
lines.count()

54223

Use `map()` to capitalise each line in the RDD, and print the first 15 capitalized lines.

In [20]:
# Capitalize each line in the RDD
capitalized = lines.map(lambda line: line.upper())
capitalized.take(15)

['                                      1869',
 '                                 WAR AND PEACE',
 '                                 BY LEO TOLSTOY',
 'BK1',
 '                                 BOOK ONE: 1805',
 'BK1|CH1',
 '  CHAPTER I',
 '',
 '  "WELL, PRINCE, SO GENOA AND LUCCA ARE NOW JUST FAMILY ESTATES OF THE',
 "BUONAPARTES. BUT I WARN YOU, IF YOU DON'T TELL ME THAT THIS MEANS WAR,",
 'IF YOU STILL TRY TO DEFEND THE INFAMIES AND HORRORS PERPETRATED BY',
 'THAT ANTICHRIST- I REALLY BELIEVE HE IS ANTICHRIST- I WILL HAVE',
 'NOTHING MORE TO DO WITH YOU AND YOU ARE NO LONGER MY FRIEND, NO LONGER',
 "MY 'FAITHFUL SLAVE,' AS YOU CALL YOURSELF! BUT HOW DO YOU DO? I SEE",
 'I HAVE FRIGHTENED YOU- SIT DOWN AND TELL ME ALL THE NEWS."']

In [22]:
lines.take(15)

['                                      1869',
 '                                 WAR AND PEACE',
 '                                 by Leo Tolstoy',
 'BK1',
 '                                 BOOK ONE: 1805',
 'BK1|CH1',
 '  CHAPTER I',
 '',
 '  "Well, Prince, so Genoa and Lucca are now just family estates of the',
 "Buonapartes. But I warn you, if you don't tell me that this means war,",
 'if you still try to defend the infamies and horrors perpetrated by',
 'that Antichrist- I really believe he is Antichrist- I will have',
 'nothing more to do with you and you are no longer my friend, no longer',
 "my 'faithful slave,' as you call yourself! But how do you do? I see",
 'I have frightened you- sit down and tell me all the news."']

Use `flatMap()` to create an RDD of the words in War and Peace and count the number of words.

In [44]:
# Split each line into words using get_words()
words = lines.flatMap(lambda line: get_words(line))
words.count()

573322

Finally, use `distinct()` to count the number of different words in the RDD.

In [58]:
# Count the number of distinct words
distinctWords = words.distinct()
distinctWords.count()

19206

### Learning activity: Set like transformations

Use the functions `union()` and `intersection()` to create RDDs of lines with either war or peace mentioned, and both war and peace being mentioned. Count how many lines of each type there are and print some examples.

In [43]:
warOrPeaceMentioned = warFiltered.union(peaceFiltered)
warAndPeaceMentioned = warFiltered.intersection(peaceFiltered)

print('Either war or peace mentioned: {} lines'.format(warOrPeaceMentioned.count()))
print('Both war and peace mentioned: {} lines'.format(warAndPeaceMentioned.count()))

Either war or peace mentioned: 1386 lines
Both war and peace mentioned: 10 lines


### Learning activity: `reduce()`

You have already seen three actions: `collect()` which returns all elements in the RDD, `take(n)`, which return the first `n` elements of the RDD, and `count()` which returns the number of elements in the RDD.

The action `reduce()` takes as input a function which collapses two elements into one. Use it to find the longest word in War and Peace.

In [46]:
words.reduce(lambda a, b: a if len(a) > len(b) else b)

'characteristically'

### Bonus activity: merging filters

Find all the lines that mention both war and peace **without** using `intersection()`.

In [47]:
mergedFilter = lines.filter(lambda line: 'war' in line.lower() and 'peace' in line.lower())
mergedFilter.count()

10

### Bonus learning activity: Finding proper nouns

The Python function `str.istitle()` returns `True` if the string `str` is titlecased: the first character is uppercase and others are lowercase. Use it to:
* Find the set of distinct words in War and Peace which are titlecased
* Find the set of distinct words in War and Peace which are not titlecased

In [56]:
'Introduction To Bigdata'.istitle()

True

In [57]:
'Introduction to Bigdata'.istitle()

False

In [61]:
titlecasedDistinct = distinctWords.filter(lambda word: word.istitle())
notTitlecasedDistinct = distinctWords.subtract(titlecasedDistinct)
print('Distinct {} words that are:'.format(distinctWords.count()))
print('Titlecased:', titlecasedDistinct.count())
print('Not titlecased:', notTitlecasedDistinct.count())
print('Sum: {}'.format(titlecasedDistinct.count() + distinctWords.filter(lambda word: not word.istitle()).count()))

Distinct 19206 words that are:
Titlecased: 3068
Not titlecased: 16138
Sum: 19206


The Python function `str.lower` returns a string with all characters of `str` lowercase. Use it, along with your previously generated RDD to find the set of words in War and Peace which only appear titlecased.

In [63]:
# First transform the RDDs to lower case words
titlecasedLowered = titlecasedDistinct.map(lambda word: word.lower())
notTitlecasedLowered = notTitlecasedDistinct.map(lambda word: word.lower())

# Then subtract
onlyTitlecased = titlecasedLowered.subtract(notTitlecasedLowered)
print('Words that only appear titlecased:',onlyTitlecased.count())
print(onlyTitlecased.take(10))

Words that only appear titlecased: 1541
['leo', 'buonaparte', 'austria', 'prussian', 'bolkonskaya', 'enghien', 'dio', 'france', 'semenov', 'rostovs']


# Key/Value pairs in Spark

### Learning activity: WordCount in Spark

Use the functions `flatMap()` and `reduceByKey()` to count the number of occurences of each word in War and Peace, and print the count of five words.

In [66]:
wordCount = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
wordCount.take(5)

[('PEACE', 1), ('Leo', 1), ('BOOK', 15), ('ONE', 2), ('1805', 24)]

### Learning activity: using `groupByKey()`

Reimplement the above word count using `groupByKey()` instead of `reduceByKey()`

In [70]:
wordCountGroupByKey = words.map(lambda word: (word, 1)).groupByKey().map(lambda keys: (keys[0], len(keys[1])))
wordCountGroupByKey.take(5)

[('PEACE', 1), ('Leo', 1), ('BOOK', 15), ('ONE', 2), ('1805', 24)]

### Bonus learning activity: computing the average of each key

The pair RDD defined below `word_line_pairs` has an element for each line in War and Peace with as key the first word, and as value the line itself. Use it to compute the average length of each line for each starting word.

In [71]:
word_line_pairs = lines.filter(lambda line: len(get_words(line)) > 0).map(lambda line: (get_words(line)[0], line))     

In [72]:
averageLength = word_line_pairs.groupByKey().map(lambda pair: (pair[0], sum([len(line) for line in pair[1]]) / len(pair[1])))
averageLength.take(5)

[('BOOK', 48.46666666666667),
 ('It', 62.230496453900706),
 ('Fedorovna', 68.0),
 ('of', 59.47846889952153),
 ('reception', 66.0)]